<a href="https://colab.research.google.com/github/dimna21/ML_Assignment4/blob/main/FER2013.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset

# 1) Load the CSV
csv_path = "/content/drive/MyDrive/FER_data/fer2013/fer2013.csv"
df = pd.read_csv(csv_path)

# 2) Split by Usage
df_train = df[df['Usage']=="Training"].copy()
df_val   = df[df['Usage']=="PublicTest"].copy()
df_test  = df[df['Usage']=="PrivateTest"].copy()

In [3]:
# 3) Balance function: upsample & random ±10 intensity shifts
def balance_dataset(df, target_count, img_shape=(48,48)):
    def augment(pix_str):
        arr = np.fromstring(pix_str, sep=' ', dtype=int).reshape(img_shape)
        shift = np.random.randint(-10, 11)
        arr = np.clip(arr + shift, 0, 255).astype(int)
        return ' '.join(map(str, arr.ravel()))
    parts = [df]
    for emo, grp in df.groupby('emotion'):
        n = len(grp)
        if n < target_count:
            extra = grp.sample(n=target_count-n, replace=True).copy()
            extra['pixels'] = extra['pixels'].map(augment)
            parts.append(extra)
    return pd.concat(parts, ignore_index=True)

max_count = df_train['emotion'].value_counts().max()  # 8989
balanced_train = balance_dataset(df_train, target_count=max_count)

In [4]:
# 4) Dataset class
class FERDataset(Dataset):
    def __init__(self, dataframe):
        self.pixels = dataframe['pixels'].values
        self.labels = dataframe['emotion'].values.astype(int)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        arr = np.fromstring(self.pixels[idx], sep=' ', dtype=np.uint8).reshape(48,48)
        arr = arr.astype(np.float32) / 255.0
        tensor = torch.from_numpy(arr).unsqueeze(0)  # shape [1,48,48]
        return tensor, self.labels[idx]


In [7]:
# 5) Simple conv-based net: 5×Conv3×3 → Dropout → FC → FC
class BaselineModel(nn.Module):
    def __init__(self,
                 in_channels: int = 1,
                 conv_channels: int = 32,
                 hidden_dim: int    = 256,
                 num_classes: int   = 7,
                 dropout_p: float   = 0.5):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels,   conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(conv_channels, conv_channels, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
        )
        self.dropout = nn.Dropout(p=dropout_p)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(conv_channels * 48 * 48, hidden_dim)
        self.relu = nn.ReLU(inplace=True)
        self.fc2 = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.convs(x)          # [B, conv_channels, 48, 48]
        x = self.dropout(x)        # dropout on feature maps
        x = self.flatten(x)        # [B, conv_channels*48*48]
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)            # [B, num_classes]
        return x

# Example instantiation
model = BaselineModel()
print(model)

BaselineModel(
  (convs): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): ReLU(inplace=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=73728, out_features=256, bias=True)
  (relu): ReLU(inplace=True)
  (fc2): Linear(in_features=256, out_features=7, bias=True)
)


In [9]:
# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [11]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# DataLoaders
batch_size = 128
train_ds = FERDataset(balanced_train)
val_ds   = FERDataset(df_val)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=2)
val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=2)


In [10]:
import torch
from tqdm import tqdm

def train_model(model, train_loader, val_loader, criterion, optimizer, device, epochs=5):
    """
    Trains and validates the given model.

    Args:
        model (nn.Module): your PyTorch model
        train_loader (DataLoader): training DataLoader
        val_loader (DataLoader): validation DataLoader
        criterion: loss function (e.g., nn.CrossEntropyLoss())
        optimizer: optimizer (e.g., torch.optim.Adam)
        device: torch.device ("cuda" or "cpu")
        epochs (int): number of epochs to train

    Returns:
        dict: history with keys 'train_loss', 'train_acc', 'val_loss', 'val_acc'
    """
    history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

    model.to(device)
    for epoch in range(1, epochs + 1):
        # Training
        model.train()
        running_loss = 0.0
        running_correct = 0
        for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch} [Train]"):
            imgs, labels = imgs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            running_correct += (outputs.argmax(dim=1) == labels).sum().item()

        train_loss = running_loss / len(train_loader.dataset)
        train_acc = running_correct / len(train_loader.dataset)
        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)

        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        with torch.no_grad():
            for imgs, labels in tqdm(val_loader, desc=f"Epoch {epoch} [Val]"):
                imgs, labels = imgs.to(device), labels.to(device)
                outputs = model(imgs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * imgs.size(0)
                val_correct += (outputs.argmax(dim=1) == labels).sum().item()

        val_loss /= len(val_loader.dataset)
        val_acc = val_correct / len(val_loader.dataset)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)

        print(
            f"Epoch {epoch}/{epochs} — "
            f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
            f"Val loss: {val_loss:.4f}, acc: {val_acc:.4f}"
        )

    return history


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BaselineModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
history = train_model(model, train_dl, val_dl, criterion, optimizer, device, epochs=5)